https://stackoverflow.com/questions/78609617/huggingface-transformer-train-function-throwing-device-in-mac-m1

# Fine Tune Phi3 for Text2SQL 

Este notebook demonstra como ajustar o modelo Phi3 para a tarefa de Text2SQL.

# Instalação de Dependências

Nesta célula, instalamos todas as bibliotecas necessárias para o ajuste fino do modelo. As bibliotecas incluem bitsandbytes, transformers, peft, accelerate, datasets, trl, entre outras.


# Verificação das Versões das Bibliotecas

Após a instalação, verificamos as versões das bibliotecas para garantir que foram instaladas corretamente.

In [1]:
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import torch
import bitsandbytes
import peft
import accelerate
import datasets
import trl


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("torch version:", torch.__version__)
print("bitsandbytes version:", bitsandbytes.__version__)
print("peft version:", peft.__version__)
print("accelerate version:", accelerate.__version__)
print("datasets version:", datasets.__version__)
print("trl version:", trl.__version__)
print(f"Device name: '{torch.cuda.get_device_name()}'")
print("Device:", device)
print(f"Device properties: '{torch.cuda.get_device_properties(torch.cuda.current_device())}'")
print("Suporta bfloat16." if torch.cuda.is_bf16_supported() else "Não suporta bfloat16.")

torch version: 2.3.1
bitsandbytes version: 0.43.3
peft version: 0.12.0
accelerate version: 0.34.2
datasets version: 2.21.0
trl version: 0.10.1
Device name: 'NVIDIA GeForce RTX 4060 Ti'
Device: cuda
Device properties: '_CudaDeviceProperties(name='NVIDIA GeForce RTX 4060 Ti', major=8, minor=9, total_memory=16059MB, multi_processor_count=34)'
Suporta bfloat16.


In [3]:
!nvidia-smi

Sun Dec 22 21:54:28 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.120                Driver Version: 550.120        CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 Ti     Off |   00000000:01:00.0  On |                  N/A |
|  0%   42C    P8              5W /  165W |     541MiB /  16380MiB |      6%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Importação de Bibliotecas e Configuração de Diretórios

Esta célula importa as bibliotecas necessárias para a análise de dados e configura os diretórios de entrada e saída. 


In [4]:
import os
from random import randrange

import torch
import numpy as np
import pandas as pd
from huggingface_hub import login
from datasets import load_dataset, Dataset


from trl import SFTConfig, SFTTrainer
from peft import LoraConfig, prepare_model_for_kbit_training, TaskType, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    TrainerCallback,
    set_seed,
    pipeline,
    TrainerCallback,
    TrainerControl,
    TrainerState,
)

# Configurações

Vamos setar algumas variáveis de ambiente, algumas secret keys e o model id à ser utilizado.


In [5]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# os.environ['TOKENIZERS_PARALLELISM'] = 'true'
# os.environ['TORCH_USE_CUDA_DSA'] = "1"

model_name = "Qwen/Qwen2-0.5B"
LOCAL_MODELPATH = "data/" + model_name.lower().replace("/","-").replace(".","_")
login(token=os.environ.get("HUGGINGFACE_TOKEN"))
set_seed(1234)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
import logging
import warnings

# Suprimir avisos específicos de FutureWarning e UserWarning
warnings.filterwarnings("ignore", message=".*resume_download.*deprecated.*", category=FutureWarning)
warnings.filterwarnings("ignore", message=".*use_cache=True.*", category=UserWarning)
warnings.filterwarnings("ignore", message=".*use_reentrant parameter should be passed explicitly.*", category=UserWarning)

# Configurar o nível de log para a biblioteca transformers
logging.getLogger("transformers.trainer").setLevel(logging.WARNING)
logging.getLogger("transformers.trainer_utils").setLevel(logging.WARNING)
logging.getLogger("transformers.training_args").setLevel(logging.WARNING)


# Funções

In [7]:
def get_latest_checkpoint(checkpoint_dir):
    try:
        checkpoints = [os.path.join(checkpoint_dir, d) for d in os.listdir(checkpoint_dir) if d.startswith("checkpoint-")]
        if not checkpoints:
            return None
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        return latest_checkpoint

    except FileNotFoundError:
        return None

# Model

In [10]:
if torch.cuda.get_device_name() == 'NVIDIA GeForce RTX 2060 SUPER':
    compute_dtype = torch.float16
    attn_implementation = 'eager'
elif torch.cuda.is_bf16_supported():
    compute_dtype = torch.bfloat16
    attn_implementation = 'flash_attention_2'
else:
    compute_dtype = torch.float16
    attn_implementation = 'eager'

print(attn_implementation)
print(compute_dtype)

flash_attention_2
torch.bfloat16


### Tokenizador

In [11]:
# tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, add_eos_token=True, use_fast=True, device_map="auto")
# tokenizer.pad_token = tokenizer.unk_token
# tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)



tokenizer.padding_side = 'right'

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

### Language Model

In [12]:
# A quantização é uma técnica para reduzir o tamanho do modelo e aumentar a eficiência computacional.
# Utilizamos a classe BitsAndBytesConfig para configurar a quantização em 4 bits, o que reduz o uso de memória e acelera o treinamento.
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype="bfloat16",
        bnb_4bit_use_double_quant=True,
)

# Usamos a classe AutoModelForCausalLM para carregar um modelo pré-treinado adequado para modelagem de linguagem causal.
# Parâmetros importantes incluem:
#  - torch_dtype=compute_dtype: Define o tipo de dado para o modelo.
#  - quantization_config=bnb_config: Aplica a configuração de quantização.
#  - device_map="auto": Distribui automaticamente o modelo nos dispositivos disponíveis.
#  - attn_implementation=attn_implementation: Define a implementação da atenção.
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=compute_dtype,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation,
)

# adapta o modelo para o treinamento em k-bits, otimizando ainda mais o desempenho.
model = prepare_model_for_kbit_training(model)

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

### Model Test

In [16]:
class LanguageModel:

    def __init__(self, tokenizer, model, device):
        self.tokenizer = tokenizer
        self.model = model
        self.device = device
        if self.tokenizer.pad_token_id is None:
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

    def tokenize(self, messages):
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        model_inputs = tokenizer([text], return_tensors="pt").to(self.device)
        return model_inputs

    def generate(self, messages):
        model_inputs = self.tokenize(messages)
        model_inputs['attention_mask'] = model_inputs['attention_mask'].to(model_inputs['input_ids'].device)
        generated_ids = model.generate(
            model_inputs.input_ids,
            max_new_tokens=512,
            do_sample=True,
            attention_mask=model_inputs['attention_mask'],
            pad_token_id=self.tokenizer.pad_token_id
        )
        generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]
        return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [15]:
%%time
llm = LanguageModel(tokenizer, model, device="cuda")

prompt = "Quem é Pelé?"

messages = [
    # {"role": "user", "content": "Olá. Você é um expert em geografia e vai me ajudar a responder algumas questões."},
    # {"role": "assistent", "content": "Tudo bem! Como posso ajudar?"},
    {"role": "user", "content": prompt},
]

llm.generate(messages)

CPU times: user 22.6 s, sys: 0 ns, total: 22.6 s
Wall time: 22.5 s


'Pelé is an Italian-American professional boxer who represented Brazil in the 1980 Olympics, where he represented Brazil at its biggest sporting event in his native country. Pelé is one of the biggest names in the world of professional boxing, having fought in several tournaments around the world since he was born on February 3, 1966. He is also known for representing Brazil in the Olympics, where he won the silver medal in the super-heavyweight division at the 1980 Summer Olympics in Moscow. Pelé\'s impressive career includes a record of more than 28 international fights, 182 wins and two losses, multiple knockout finishes, and many successful defenses. His victories are widely regarded as some of the greatest fights in professional boxing history. Pelé has been inducted into the International Boxing Hall of Fame and the Ring of Honor Hall of Fame, among other professional boxing bodies. \n\nOne of the most talked-about topics surrounding Pelé in recent years has been his ongoing feud

# Dataset

In [17]:
def format_dataset_chatml(row):
    messages = [
        {
            "content": f"Pergunta: {row['pergunta']}\nContexto: {row['contexto']}",
            "role": "user"
        },
        {
            "content": f"{row['resposta']}",
            "role": "assistant"
        }
    ]

    return {"text": tokenizer.apply_chat_template(messages, add_generation_prompt=False, tokenize=False)}

dataset = load_dataset("emdemor/sql-create-context-pt", split="train").shuffle(seed=42).select(range(200))
dataset_chatml = dataset.map(format_dataset_chatml).train_test_split(test_size=0.10, seed=1234)
dataset_chatml

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['pergunta', 'contexto', 'resposta', 'text'],
        num_rows: 180
    })
    test: Dataset({
        features: ['pergunta', 'contexto', 'resposta', 'text'],
        num_rows: 20
    })
})

In [19]:
dataset_chatml["train"][0]

{'pergunta': 'Que equipa visitante terminou com um resultado final de 18-40?',
 'contexto': 'CREATE TABLE table_name_77 (visiting_team VARCHAR, final_score VARCHAR)',
 'resposta': 'SELECT visiting_team FROM table_name_77 WHERE final_score = "18-40"',
 'text': '<|im_start|>system\nYou are a helpful assistant<|im_end|>\n<|im_start|>user\nPergunta: Que equipa visitante terminou com um resultado final de 18-40?\nContexto: CREATE TABLE table_name_77 (visiting_team VARCHAR, final_score VARCHAR)<|im_end|>\n<|im_start|>assistant\nSELECT visiting_team FROM table_name_77 WHERE final_score = "18-40"<|im_end|>\n'}

# Training

`SFTTrainer` é uma classe que simplifica o processo de fine-tuning de modelos de linguagem. Ela gerencia o treinamento e avaliação do modelo, tratando de diversos detalhes técnicos que normalmente exigiriam muito código para serem configurados manualmente.

### Principais Funcionalidades

1. **Configuração do Treinamento**:
   - Facilita a configuração de hiperparâmetros como taxa de aprendizado, tamanho do lote, número de épocas, etc.
   - Permite a integração com frameworks de logging como WandB, TensorBoard, entre outros.

2. **Gerenciamento de Dados**:
   - Gerencia o carregamento e pré-processamento dos dados de treinamento e validação.
   - Suporta diferentes formatos de dados e tokenizadores.

3. **Treinamento e Avaliação**:
   - Implementa loops de treinamento e avaliação, com suporte a múltiplas GPUs e distribuído.
   - Realiza checkpoints automáticos do modelo, salvando os melhores resultados com base em métricas de avaliação.


#### Configuração do `SFTConfig`
A classe `SFTConfig` é usada para definir diversas configurações para o treinamento do modelo. Aqui estão os parâmetros principais:

- `dataset_text_field`: Campo de texto do dataset.
- `max_seq_length`: Comprimento máximo da sequência.
- `output_dir`: Diretório onde o modelo treinado será salvo.
- `eval_strategy`: Estratégia de avaliação, como "steps" (passos) ou "epoch" (épocas).
- `do_eval`: Se a avaliação deve ser feita durante o treinamento.
- `optim`: Otimizador a ser usado, neste caso `adamw_torch`.
- `per_device_train_batch_size`: Tamanho do lote de treinamento por dispositivo.
- `gradient_accumulation_steps`: Número de passos de acumulação de gradientes antes de realizar uma atualização do modelo.
- `per_device_eval_batch_size`: Tamanho do lote de avaliação por dispositivo.
- `log_level`: Nível de log, neste caso "debug".
- `save_strategy`: Estratégia de salvamento, como "steps" ou "epoch".
- `logging_steps`: Número de passos entre registros de log.
- `learning_rate`: Taxa de aprendizado.
- `fp16`: Se deve usar precisão mista (16 bits).
- `bf16`: Se deve usar bfloat16, dependendo do suporte CUDA.
- `eval_steps`: Número de passos entre avaliações.
- `num_train_epochs`: Número de épocas de treinamento.
- `warmup_ratio`: Proporção de warmup (aquecimento) da taxa de aprendizado.
- `lr_scheduler_type`: Tipo de agendador de taxa de aprendizado, neste caso "linear". <a href="#01">[Apêndice A]</a>
- `report_to`: Onde reportar os resultados do treinamento, como "none" (nenhum).
- `seed`: Semente para replicabilidade.

#### Configuração do `LoraConfig`
A classe `LoraConfig` é usada para definir configurações específicas de Low-Rank Adaptation (LoRA):

- `r`: Parâmetro de rank da decomposição.
- `lora_alpha`: Parâmetro de escala de LoRA.
- `lora_dropout`: Dropout aplicado nas camadas LoRA.
- `task_type`: Tipo de tarefa, como `TaskType.CAUSAL_LM`.
- `target_modules`: Módulos alvo para LoRA.

#### Inicialização do `SFTTrainer`
A classe `SFTTrainer` é inicializada com as seguintes configurações:

- `model`: O modelo a ser treinado.
- `train_dataset`: Dataset de treinamento.
- `eval_dataset`: Dataset de avaliação.
- `args`: Configurações de treinamento (instância de `SFTConfig`).
- `peft_config`: Configurações de LoRA (instância de `LoraConfig`).
- `tokenizer`: Tokenizador a ser usado.

### Parâmetros para Mudar a Frequência de Checkpoints

Para alterar a frequência de checkpoints, você pode ajustar os seguintes parâmetros em `SFTConfig`:

1. **Salvar Checkpoints por Passos**:
   - **Parâmetro**: `save_strategy`
   - **Valor**: "steps"
   - **Parâmetro Adicional**: `save_steps`
   - **Descrição**: Define a frequência de salvamento em passos.
   - **Exemplo**:
     ```python
     sft_config = SFTConfig(
         ...,
         save_strategy="steps",
         save_steps=100,  # Salvando a cada 100 passos
         ...
     )
     ```

2. **Salvar Checkpoints por Épocas**:
   - **Parâmetro**: `save_strategy`
   - **Valor**: "epoch"
   - **Descrição**: Define que o salvamento será feito ao final de cada época.
   - **Exemplo**:
     ```python
     sft_config = SFTConfig(
         ...,
         save_strategy="epoch",  # Salvando ao final de cada época
         ...
     )
     ```

3. **Frequência de Avaliação**:
   - **Parâmetro**: `eval_steps`
   - **Descrição**: Define a frequência de avaliações em passos.
   - **Exemplo**:
     ```python
     sft_config = SFTConfig(
         ...,
         eval_steps=50,  # Avaliando a cada 50 passos
         ...
     )
     ```


#### Parametros de treinamento

In [20]:
%%time
output_dir = "models/FT-01"

sft_config = SFTConfig(
    seed=42,
    output_dir=output_dir,
    dataset_text_field="text",
    max_seq_length=512,

    # Training Hyperparameters
    learning_rate=1e-4,
    num_train_epochs=3,
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
    
    # Validation 
    do_eval=True,
    eval_strategy="steps",
    eval_steps=20,

    # Chackpoints
    save_strategy="steps",  # Salvando a cada 100 passos
    save_steps=10,         # Salvando a cada 100 passos
    # save_strategy="epoch",  # Salvando ao final de cada época

    # Loggings
    log_level="warning",
    logging_steps=20,

    # Weights yype
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),

    # Report to Weights And Bias? TensorBoard?
    report_to="none",
)

peft_config = LoraConfig(
        r=16,
        lora_alpha=16,
        lora_dropout=0.05,
        task_type=TaskType.CAUSAL_LM,
        target_modules=['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"],
)

CPU times: user 2.19 ms, sys: 0 ns, total: 2.19 ms
Wall time: 13.1 ms


#### Treinamento

In [21]:
class SaveCheckpointCallback(TrainerCallback):
    def on_save(self, args, state, control, **kwargs):
        print(f"Saving checkpoint at step {state.global_step}")



trainer = SFTTrainer(
    model,
    train_dataset=dataset_chatml['train'],
    eval_dataset=dataset_chatml['test'],
    args=sft_config,
    peft_config=peft_config,
    tokenizer=tokenizer,
    callbacks=[SaveCheckpointCallback()],
)

trainer.train()

trainer.save_model()

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss
20,1.962500,1.232542
40,1.084500,1.069918
60,1.007000,1.057704


Saving checkpoint at step 10


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Saving checkpoint at step 20
Saving checkpoint at step 30
Saving checkpoint at step 40
Saving checkpoint at step 50
Saving checkpoint at step 60
Saving checkpoint at step 69


#### Treinamento com EarlyStopping

In [13]:
sft_config = SFTConfig(
    seed=42,
    output_dir=output_dir,
    dataset_text_field="text",
    max_seq_length=512,

    # Training Hyperparameters
    learning_rate=1e-4,
    num_train_epochs=3,
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
    
    # Validation 
    do_eval=True,
    eval_strategy="steps",
    eval_steps=2,

    # Chackpoints
    save_strategy="steps",  # Salvando a cada 100 passos
    save_steps=10,         # Salvando a cada 100 passos
    # save_strategy="epoch",  # Salvando ao final de cada época

    # Loggings
    log_level="warning",
    logging_steps=20,

    # Weights yype
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),

    # Report to Weights And Bias? TensorBoard?
    report_to="none",
)


In [14]:
class SaveCheckpointCallback(TrainerCallback):
    def on_save(self, args, state, control, **kwargs):
        print(f"Saving checkpoint at step {state.global_step}")

class EarlyStoppingCallback(TrainerCallback):
    def __init__(self, early_stopping_patience=3, early_stopping_threshold=0.02):
        self.early_stopping_patience = early_stopping_patience
        self.early_stopping_threshold = early_stopping_threshold
        self.best_metric = None
        self.counter = 0

    def on_evaluate(self, args, state: TrainerState, control: TrainerControl, metrics=None, **kwargs):
        current_metric = metrics.get("eval_loss")  # Use the relevant metric for your task

        if current_metric is None:
            return

        if self.best_metric is None or current_metric < self.best_metric - self.early_stopping_threshold:
            self.best_metric = current_metric
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.early_stopping_patience:
                control.should_training_stop = True
                print(f"Early stopping at step {state.global_step} with best eval_loss = {self.best_metric}")



trainer = SFTTrainer(
    model,
    train_dataset=dataset_chatml['train'],
    eval_dataset=dataset_chatml['test'],
    args=sft_config,
    peft_config=peft_config,
    tokenizer=tokenizer,
    callbacks=[SaveCheckpointCallback(), EarlyStoppingCallback( early_stopping_threshold=0.3)],
)

trainer.train()

trainer.save_model()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss
2,No log,2.968371
4,No log,2.711381
6,No log,2.366761
8,No log,2.010422
10,No log,1.728735
12,No log,1.505082
14,No log,1.397608
16,No log,1.329260
18,No log,1.278901


Saving checkpoint at step 10
Early stopping at step 18 with best eval_loss = 1.505082368850708


## Continuar a partir de um checkpoint

In [10]:
output_dir = "models/FT-01"
latest_checkpoint = get_latest_checkpoint(output_dir)

tokenizer = AutoTokenizer.from_pretrained(latest_checkpoint)
# model = AutoModelForCausalLM.from_pretrained(latest_checkpoint)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
def format_dataset_chatml(row):
    messages = [
        {
            "content": f"Pergunta: {row['pergunta']}\nContexto: {row['contexto']}",
            "role": "user"
        },
        {
            "content": f"{row['resposta']}",
            "role": "assistant"
        }
    ]

    return {"text": tokenizer.apply_chat_template(messages, add_generation_prompt=False, tokenize=False)}

dataset = load_dataset("emdemor/sql-create-context-pt", split="train").shuffle(seed=42).select(range(200))
dataset_chatml = dataset.map(format_dataset_chatml).train_test_split(test_size=0.10, seed=1234)
dataset_chatml

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['pergunta', 'contexto', 'resposta', 'text'],
        num_rows: 180
    })
    test: Dataset({
        features: ['pergunta', 'contexto', 'resposta', 'text'],
        num_rows: 20
    })
})

In [12]:
sft_config = SFTConfig(
    seed=42,
    output_dir=output_dir,
    dataset_text_field="text",
    max_seq_length=512,

    # Training Hyperparameters
    learning_rate=1e-4,
    num_train_epochs=3,
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
    
    # Validation 
    do_eval=True,
    eval_strategy="steps",
    eval_steps=20,

    # Chackpoints
    save_strategy="steps",  # Salvando a cada 100 passos
    save_steps=10,         # Salvando a cada 100 passos
    # save_strategy="epoch",  # Salvando ao final de cada época

    # Loggings
    log_level="warning",
    logging_steps=20,

    # Weights yype
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),

    # Report to Weights And Bias?
    report_to="none",
)

peft_config = LoraConfig(
        r=16,
        lora_alpha=16,
        lora_dropout=0.05,
        task_type=TaskType.CAUSAL_LM,
        target_modules=['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"],
)


class SaveCheckpointCallback(TrainerCallback):
    def on_save(self, args, state, control, **kwargs):
        print(f"Saving checkpoint at step {state.global_step}")


trainer = SFTTrainer(
    latest_checkpoint,
    train_dataset=dataset_chatml['train'],
    eval_dataset=dataset_chatml['test'],
    args=sft_config,
    peft_config=peft_config,
    tokenizer=tokenizer,
    callbacks=[SaveCheckpointCallback()],
)


trainer.train(resume_from_checkpoint=latest_checkpoint)

trainer.save_model()

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:174: UserWarning: You passed a model_id to the SFTTrainer. This will automatically create an `AutoModelForCausalLM` or a `PeftModel` (if you passed a `peft_config`) for you.
  warnings.warn(


Map:   0%|          | 0/180 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

# Apendices

<a id="#01"></a>

## A. Parâmetro `lr_scheduler_type`

O parâmetro `lr_scheduler_type` é usado para especificar o tipo de agendador de taxa de aprendizado (Learning Rate Scheduler) a ser utilizado durante o treinamento do modelo. Um agendador de taxa de aprendizado ajusta a taxa de aprendizado ao longo do treinamento, o que pode ajudar a melhorar a convergência e o desempenho do modelo.



A taxa de aprendizado é um dos hiperparâmetros mais importantes no treinamento de modelos de aprendizado de máquina. Ajustá-la dinamicamente durante o treinamento pode ajudar o modelo a aprender mais rapidamente no início e a estabilizar conforme se aproxima do mínimo da função de perda.

Existem vários tipos de agendadores de taxa de aprendizado, e cada um tem uma estratégia diferente para ajustar a taxa de aprendizado ao longo do tempo. Abaixo, descrevo alguns dos tipos comuns de agendadores que podem ser especificados pelo parâmetro `lr_scheduler_type`:

1. **Linear**:
   - **Descrição**: Diminui a taxa de aprendizado de forma linear ao longo do treinamento.
   - **Uso**: `linear`
   - **Comportamento**: Começa com a taxa de aprendizado inicial e a diminui gradualmente até zero ao final do treinamento.
   - **Exemplo**: Ideal para cenários onde se deseja uma diminuição constante da taxa de aprendizado.

2. **Cosine**:
   - **Descrição**: Usa uma função cosseno para diminuir a taxa de aprendizado.
   - **Uso**: `cosine`
   - **Comportamento**: A taxa de aprendizado diminui de forma não-linear, com uma curva suave.
   - **Exemplo**: Útil para evitar grandes mudanças bruscas na taxa de aprendizado, promovendo uma convergência mais suave.

3. **Cosine with Restarts**:
   - **Descrição**: Variante do agendador cosseno, mas com reinicializações periódicas.
   - **Uso**: `cosine_with_restarts`
   - **Comportamento**: A taxa de aprendizado segue uma função cosseno, mas reinicia para a taxa inicial em intervalos definidos.
   - **Exemplo**: Eficaz para evitar ficar preso em mínimos locais.

4. **Polynomial**:
   - **Descrição**: Diminui a taxa de aprendizado de acordo com uma função polinomial.
   - **Uso**: `polynomial`
   - **Comportamento**: A taxa de aprendizado decai de acordo com uma potência especificada.
   - **Exemplo**: Fornece um controle mais preciso sobre o decaimento da taxa de aprendizado.

5. **Constant**:
   - **Descrição**: Mantém a taxa de aprendizado constante ao longo do treinamento.
   - **Uso**: `constant`
   - **Comportamento**: Não ajusta a taxa de aprendizado.
   - **Exemplo**: Útil em cenários onde a taxa de aprendizado não precisa mudar.

6. **Constant with Warmup**:
   - **Descrição**: Mantém a taxa de aprendizado constante após um período inicial de aquecimento.
   - **Uso**: `constant_with_warmup`
   - **Comportamento**: Começa com uma taxa de aprendizado baixa e a aumenta gradualmente até a taxa desejada, mantendo-a constante após o aquecimento.
   - **Exemplo**: Bom para estabilizar o treinamento no início.

7. **Exponential**:
   - **Descrição**: Diminui a taxa de aprendizado exponencialmente.
   - **Uso**: `exponential`
   - **Comportamento**: A taxa de aprendizado decresce de forma exponencial.
   - **Exemplo**: Útil quando se deseja uma diminuição rápida da taxa de aprendizado.

### Exemplo de Uso

Aqui está um exemplo de configuração do `SFTConfig` usando o agendador de taxa de aprendizado linear:

```python
sft_config = SFTConfig(
    dataset_text_field="text",
    max_seq_length=512,
    output_dir=LOCAL_MODELPATH,
    eval_strategy="steps",
    do_eval=True,
    optim="adamw_torch",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=8,
    log_level="debug",
    save_strategy="steps",
    save_steps=100,
    logging_steps=20,
    learning_rate=1e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    eval_steps=50,
    num_train_epochs=10,
    warmup_ratio=0.1,
    lr_scheduler_type="linear",  # Tipo de agendador de taxa de aprendizado
    report_to="none",
    seed=42,
)
```

### Conclusão

O `lr_scheduler_type` permite controlar como a taxa de aprendizado será ajustada durante o treinamento, o que pode impactar significativamente a eficiência e o desempenho do modelo. Escolher o agendador correto depende da natureza do problema e das preferências específicas de treinamento.